In [163]:
import os
import pandas as pd
import numpy as np
import pyodbc
import warnings
import datetime
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from openpyxl import Workbook
import time
server = 'BOMSSPROD551.micron.com\BOMSSPROD551' 
database = 'CPG' 
username = 'CPGLOADER' 
password = 'lexCrurep2'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';UID='+username+';PWD='+ password, port=14338)
cnxn.timeout = 30
cursor = cnxn.cursor()


In [164]:
try:
    sql=pd.read_sql("""select r.*,q.[PRODUCT_GROUP]
	 from
(SELECT [RETAILER_SKU] [ASIN],  Case When WK_ID =202053 then 202101
	     --When WK_ID >=202102 then WK_ID+1
	   	 When WK_ID >=202101 and WK_ID<=202151 then WK_ID+1
		 When WK_ID =202152 then 202201
		  When WK_ID >=202201  then WK_ID+1
	 ELSE WK_ID
  End as [Week ID]

	 ,SUM(VOL) AS Volume
	 , cast(sum([ESTMTE_SALE]) as float) AS [Estimated Sales], avg(RTL_PRC) [Price]
	 , Case When UPPER(BRND_NM) = 'CRUCIAL' AND UPPER(LEFT([MODL_NUM], 2)) = 'BL' Then 'Ballistix'
			Else BRND_NM
	   End AS [Brand Name]
	, SUB_CAT_NM AS [Subcategory Name]

/** Commented on 02/15 ***/
	-- ,  Case When WK_ID =202053 then 202101
	--   When WK_ID >=202101 and WK_ID<=202151 then WK_ID+1--added 1/20/2022
	---- When WK_ID >=202101 then WK_ID+1
	-- ELSE WK_ID
 -- End as [Week ID]
 /*** end comment **/

  
/*	 , RGN_CD AS [Region Code]
	 ,TITL AS [Description]--added 3/2/2022 */
FROM cpgbi.BI_STACKLINE
where BRND_NM in ('Crucial','BL') and RGN_CD ='US'
GROUP BY [RETAILER_SKU], Case When UPPER(BRND_NM) = 'CRUCIAL' AND UPPER(LEFT([MODL_NUM], 2)) = 'BL' Then 'Ballistix'
			Else BRND_NM
	   End 
	 , SUB_CAT_NM 
	 , WK_ID
) r
inner join (	 
SELECT   A.[ASIN],concat(datepart(year,B.[EFFECTIVE_DATE]),format(datepart(WEEK,B.[EFFECTIVE_DATE]),'00')) [WEEK_ID],A.PRODUCT_GROUP,A.PRODUCT_TYPE,B.DATA_TYPE AS [DEAL TYPE],round(avg(cast(SALE_RANK as int)),0) SALE_RANK,avg(cast(OOS_PERCENT as float)) OOS_PERCENT,avg(cast(OOS_30_PERCENT as float)) OOS_30_PERCENT ,avg(cast(OOS_90_PERCENT as float)) OOS_90_PERCENT,avg(A.CURRENT_PRICE) [Current_Price],min(B.DATA_POINT) /* over (partition by A.[ASIN],concat(datepart(year,A.[LAST_PRICE_CHANGE]),datepart(WEEK,A.[LAST_PRICE_CHANGE])), A.PRODUCT_GROUP,A.PRODUCT_TYPE,B.DATA_TYPE )*/ AS [MIN_DEAL_PRICE],max(B.DATA_POINT) /* over (partition by A.[ASIN],concat(datepart(year,A.[LAST_PRICE_CHANGE]),datepart(WEEK,A.[LAST_PRICE_CHANGE])), A.PRODUCT_GROUP,A.PRODUCT_TYPE,B.DATA_TYPE )*/ AS [MAX_DEAL_PRICE],stdev(B.DATA_POINT) /*over (partition by A.[ASIN],concat(datepart(year,A.[LAST_PRICE_CHANGE]),datepart(WEEK,A.[LAST_PRICE_CHANGE])), A.PRODUCT_GROUP,A.PRODUCT_TYPE,B.DATA_TYPE )*/ AS [STD_DEAL_PRICE] FROM [cpgbi].[BI_KEEPA_FEED_DATA]  A WITH (NOLOCK)
  INNER JOIN [cpgbi].[BI_KEEPA_FEED_HISTORICAL_DATA] B WITH (NOLOCK) ON A.[KEY] = B.[KEY] where ACTIVE_FLAG='Y' AND A.COUNTRY='US' AND B.DATA_TYPE LIKE '%AMAZON%' and A.[MANUFACTURER]='CRUCIAL' 
  GROUP BY A.[ASIN],concat(datepart(year,B.[EFFECTIVE_DATE]),format(datepart(WEEK,B.[EFFECTIVE_DATE]),'00')),A.PRODUCT_GROUP,A.PRODUCT_TYPE,B.DATA_TYPE
  /*order by A.[LAST_PRICE_CHANGE] desc*/ ) q on r.ASIN=q.ASIN and r.[Week ID]=q.WEEK_ID""",cnxn)
    print(sql[0:2])    
finally:
    cursor.close()
    cnxn.close()

         ASIN   Week ID  Volume  Estimated Sales   Price Brand Name  \
0  B003J5JB12  201804.0     6.0          3092.10  515.35    Crucial   
1  B003J5JB12  201805.0     6.0          2999.94  499.99    Crucial   

              Subcategory Name PRODUCT_GROUP  
0  Internal Solid State Drives     MX500 SSD  
1  Internal Solid State Drives     MX500 SSD  


In [3]:
sql['Min_Current_Price']=sql.groupby(['ASIN'])['Price'].transform('min')
sql['Max_Current_Price']=sql.groupby(['ASIN'])['Price'].transform('max')
sql['Mean_Current_Price']=sql.groupby(['ASIN'])['Price'].transform('mean')
sql['Stdev_Current_Price']=sql.groupby(['ASIN'])['Price'].transform('std')

In [4]:
sql

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,Min_Current_Price,Max_Current_Price,Mean_Current_Price,Stdev_Current_Price
0,B00006BT1Q,201804.0,2.0,38.00,19.00,Crucial,Memory,19.00,19.00,19.0000,0.000000
1,B00006BT1Q,201805.0,2.0,38.00,19.00,Crucial,Memory,19.00,19.00,19.0000,0.000000
2,B00006BT1Q,201806.0,1.0,19.00,19.00,Crucial,Memory,19.00,19.00,19.0000,0.000000
3,B00006BT1Q,201808.0,0.0,0.00,19.00,Crucial,Memory,19.00,19.00,19.0000,0.000000
4,B00006BT1Q,201809.0,3.0,57.00,19.00,Crucial,Memory,19.00,19.00,19.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
84974,B09YZ3KZ1D,202224.0,10.0,909.80,90.98,Crucial,Internal Solid State Drives,90.98,95.98,92.4125,2.403100
84975,B09YZ3KZ1D,202225.0,16.0,1535.68,95.98,Crucial,Internal Solid State Drives,90.98,95.98,92.4125,2.403100
84976,B09YZ3X8BZ,202222.0,33.0,4577.43,138.71,Crucial,Internal Solid State Drives,138.71,150.98,142.8900,7.007446
84977,B09YZ3X8BZ,202223.0,42.0,5837.16,138.98,Crucial,Internal Solid State Drives,138.71,150.98,142.8900,7.007446


In [165]:
sql['Week ID']=sql['Week ID'].astype('str')
sql['Week ID']=sql['Week ID'].str[:6]

In [167]:
sql['trend']=sql['Week ID'].str[:4].astype(int)-2018
sql['Week']=sql['Week ID'].str[4:6]

In [168]:
sql

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week
0,B003J5JB12,201804,6.0,3092.10,515.350000,Crucial,Internal Solid State Drives,MX500 SSD,0,04
1,B003J5JB12,201805,6.0,2999.94,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,05
2,B003J5JB12,201806,12.0,5999.88,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,06
3,B003J5JB12,201807,11.0,5499.89,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,07
4,B003J5JB12,201808,10.0,4999.90,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,08
...,...,...,...,...,...,...,...,...,...,...
2938,B09S2QSTRS,202224,1.0,123.27,123.270000,Crucial,Memory,DDR5 SODIMM,4,24
2939,B09S2QT75C,202212,0.0,0.00,0.000000,Crucial,Memory,DDR5 SODIMM,4,12
2940,B09S2QT75C,202222,3.0,161.96,53.986667,Crucial,Memory,DDR5 SODIMM,4,22
2941,B09S2QT75C,202223,3.0,161.94,53.980000,Crucial,Memory,DDR5 SODIMM,4,23


In [169]:
sql['week_date'] = pd.to_datetime(
    sql['Week ID'].str[:4].astype(str) + sql['Week'].astype(str) + "1",
    format='%G%V%w'
)

In [170]:
sql

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,week_date
0,B003J5JB12,201804,6.0,3092.10,515.350000,Crucial,Internal Solid State Drives,MX500 SSD,0,04,2018-01-22
1,B003J5JB12,201805,6.0,2999.94,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,05,2018-01-29
2,B003J5JB12,201806,12.0,5999.88,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,06,2018-02-05
3,B003J5JB12,201807,11.0,5499.89,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,07,2018-02-12
4,B003J5JB12,201808,10.0,4999.90,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,08,2018-02-19
...,...,...,...,...,...,...,...,...,...,...,...
2938,B09S2QSTRS,202224,1.0,123.27,123.270000,Crucial,Memory,DDR5 SODIMM,4,24,2022-06-13
2939,B09S2QT75C,202212,0.0,0.00,0.000000,Crucial,Memory,DDR5 SODIMM,4,12,2022-03-21
2940,B09S2QT75C,202222,3.0,161.96,53.986667,Crucial,Memory,DDR5 SODIMM,4,22,2022-05-30
2941,B09S2QT75C,202223,3.0,161.94,53.980000,Crucial,Memory,DDR5 SODIMM,4,23,2022-06-06


In [171]:
sql['month']=sql['week_date'].dt.month

In [172]:
sql

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,week_date,month
0,B003J5JB12,201804,6.0,3092.10,515.350000,Crucial,Internal Solid State Drives,MX500 SSD,0,04,2018-01-22,1
1,B003J5JB12,201805,6.0,2999.94,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,05,2018-01-29,1
2,B003J5JB12,201806,12.0,5999.88,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,06,2018-02-05,2
3,B003J5JB12,201807,11.0,5499.89,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,07,2018-02-12,2
4,B003J5JB12,201808,10.0,4999.90,499.990000,Crucial,Internal Solid State Drives,MX500 SSD,0,08,2018-02-19,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2938,B09S2QSTRS,202224,1.0,123.27,123.270000,Crucial,Memory,DDR5 SODIMM,4,24,2022-06-13,6
2939,B09S2QT75C,202212,0.0,0.00,0.000000,Crucial,Memory,DDR5 SODIMM,4,12,2022-03-21,3
2940,B09S2QT75C,202222,3.0,161.96,53.986667,Crucial,Memory,DDR5 SODIMM,4,22,2022-05-30,5
2941,B09S2QT75C,202223,3.0,161.94,53.980000,Crucial,Memory,DDR5 SODIMM,4,23,2022-06-06,6


In [173]:
p=sql['ASIN'].unique()

In [174]:
q=['B086BGWNY8','B089DNM8LR','B08GVDNTGJ','B098WKQRDL','B07YD579WM','B0781VSXBP','B0786QNS9B','B078211KBB','B003J5JB12','B08W1F8YGJ','B08FSZT2J7','B08FSNKNSV','B08W1KDM9K']

In [175]:
df=pd.DataFrame()
for x in q:
    df1=sql[sql['ASIN']==x]
    df=df.append(df1)

In [176]:
df

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,week_date,month
1263,B086BGWNY8,202102,1017.0,54348.48,53.440000,Crucial,Internal Solid State Drives,P2 SSD,3,02,2021-01-11,1
1264,B086BGWNY8,202103,1064.0,57115.52,53.680000,Crucial,Internal Solid State Drives,P2 SSD,3,03,2021-01-18,1
1265,B086BGWNY8,202104,1487.0,78796.13,52.990000,Crucial,Internal Solid State Drives,P2 SSD,3,04,2021-01-25,1
1266,B086BGWNY8,202105,1136.0,60196.64,52.990000,Crucial,Internal Solid State Drives,P2 SSD,3,05,2021-02-01,2
1267,B086BGWNY8,202106,782.0,45066.66,57.630000,Crucial,Internal Solid State Drives,P2 SSD,3,06,2021-02-08,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2496,B08W1KDM9K,202221,52.0,18615.48,357.990000,Crucial,External Solid State Drives,X6,4,21,2022-05-23,5
2497,B08W1KDM9K,202222,76.0,27093.24,356.490000,Crucial,External Solid State Drives,X6,4,22,2022-05-30,5
2498,B08W1KDM9K,202223,64.0,22271.99,347.999844,Crucial,External Solid State Drives,X6,4,23,2022-06-06,6
2499,B08W1KDM9K,202224,80.0,26800.00,335.000000,Crucial,External Solid State Drives,X6,4,24,2022-06-13,6


In [177]:
df['ASIN'].unique()

array(['B086BGWNY8', 'B089DNM8LR', 'B08GVDNTGJ', 'B098WKQRDL',
       'B07YD579WM', 'B0781VSXBP', 'B0786QNS9B', 'B078211KBB',
       'B003J5JB12', 'B08W1F8YGJ', 'B08FSZT2J7', 'B08FSNKNSV',
       'B08W1KDM9K'], dtype=object)

In [178]:
df.isna().any()

ASIN                False
Week ID             False
Volume              False
Estimated Sales     False
Price               False
Brand Name          False
Subcategory Name    False
PRODUCT_GROUP       False
trend               False
Week                False
week_date           False
month               False
dtype: bool

In [212]:
df[df['STD_DEAL_PRICE'].isnull()]

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,PRODUCT_GROUP,PRODUCT_TYPE,Current_Price,MIN_DEAL_PRICE,MAX_DEAL_PRICE,STD_DEAL_PRICE,Min_Current_Price,Max_Current_Price,Mean_Current_Price,Stdev_Current_Price,trend,Week,week_date,month
1310,B086BGWNY8,202201,1095.0,49264.05,44.99,Crucial,P2 SSD,SSD,55.99,56.67,56.67,NaN,42.990000,59.99,53.681389,5.331634,2022,01,2022-01-03,1
1455,B089DNM8LR,202201,273.0,24881.22,91.14,Crucial,P2 SSD,SSD,99.99,109.99,109.99,NaN,73.489997,126.09,98.910965,10.621230,2022,01,2022-01-03,1
2348,B08GVDNTGJ,202142,101.0,20099.00,199.00,Crucial,P2 SSD,SSD,199.99,217.99,217.99,NaN,159.989984,257.99,199.965549,25.351645,2021,42,2021-10-18,10
2358,B08GVDNTGJ,202201,168.0,30191.28,179.71,Crucial,P2 SSD,SSD,199.99,204.88,204.88,NaN,159.989984,257.99,199.965549,25.351645,2022,01,2022-01-03,1
2606,B098WKQRDL,202201,94.0,27166.00,289.00,Crucial,P5 Plus,SSD,248.84,358.99,358.99,NaN,204.070005,367.99,280.512683,45.421925,2022,01,2022-01-03,1
907,B07YD579WM,202201,1486.0,124809.14,83.99,Crucial,BX500 SSD,SSD,87.99,89.99,89.99,NaN,65.060000,98.99,88.115000,6.285635,2022,01,2022-01-03,1
284,B0781VSXBP,202201,311.0,15217.23,48.93,Crucial,MX500 SSD,SSD,49.99,49.99,49.99,NaN,35.990000,79.99,52.560392,12.295049,2022,01,2022-01-03,1
594,B0786QNS9B,202201,288.0,16453.44,57.13,Crucial,MX500 SSD,SSD,59.99,59.99,59.99,NaN,52.990000,134.99,75.287516,25.483341,2022,01,2022-01-03,1
432,B078211KBB,202143,1359.0,123410.79,90.81,Crucial,MX500 SSD,SSD,99.99,90.81,90.81,NaN,81.990000,272.16,137.935032,55.557589,2021,43,2021-10-25,10
441,B078211KBB,202201,1455.0,145485.45,99.99,Crucial,MX500 SSD,SSD,99.99,99.99,99.99,NaN,81.990000,272.16,137.935032,55.557589,2022,01,2022-01-03,1


In [213]:
# Using sklearn imputer
import numpy as np 
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [214]:
dicts = {}
keys = df[df['STD_DEAL_PRICE'].isnull()]['ASIN']


In [215]:
for k in keys:
    q=df[df['STD_DEAL_PRICE'].isna() & (df['ASIN']==k)].index.values
    dicts[k]=q


In [216]:
dicts['B08FSNKNSV']

array([2198, 2209], dtype=int64)

In [217]:
for key, values in dicts.items():
    for value in values:
        imputer.fit(df[df.ASIN ==key][["ASIN","STD_DEAL_PRICE"]])
        df.at[value, 'STD_DEAL_PRICE']= imputer.transform(df[ (df.ASIN ==key) & (df['STD_DEAL_PRICE'].isna())][["ASIN","STD_DEAL_PRICE"]] )[0,1]

In [218]:
df[df['STD_DEAL_PRICE'].isnull()]

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,PRODUCT_GROUP,PRODUCT_TYPE,Current_Price,MIN_DEAL_PRICE,MAX_DEAL_PRICE,STD_DEAL_PRICE,Min_Current_Price,Max_Current_Price,Mean_Current_Price,Stdev_Current_Price,trend,Week,week_date,month


In [179]:
def check_outliers(df,features,k=5):
  data = df.copy()
  for f in features:
    # data['mean+'+str(k)+'*std_'+f] = data.groupby('sku')[f].transform(
    # lambda x: x.mean()+k*x.std()  )
    # data['mean-'+str(k)+'*std_'+f] = data.groupby('sku')[f].transform(
    # lambda x: x.mean()-k*x.std()  )
    data['outlier_'+f] = data.groupby('ASIN')[f].transform(
    lambda x: (x > (x.mean()+k*x.std()) )  |(x < (x.mean()-k*x.std()) )  )
  return(data)

In [180]:
df_1 = check_outliers(df,['Volume','Price'],5)

In [181]:
df_1[df_1.outlier_Price]

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,week_date,month,outlier_Volume,outlier_Price


In [182]:
df_1[df_1.outlier_Volume]

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,week_date,month,outlier_Volume,outlier_Price
1432,B089DNM8LR,202126,3188.0,234286.11,73.489997,Crucial,Internal Solid State Drives,P2 SSD,3,26,2021-06-28,6,True,False
2353,B08GVDNTGJ,202148,1390.0,222386.10,159.990000,Crucial,Internal Solid State Drives,P2 SSD,3,48,2021-11-29,11,True,False
2621,B098WKQRDL,202216,1893.0,386304.52,204.070005,Crucial,Internal Solid State Drives,P5 Plus,4,16,2022-04-18,4,True,False
882,B07YD579WM,202126,7999.0,520414.94,65.060000,Crucial,Internal Solid State Drives,BX500 SSD,3,26,2021-06-28,6,True,False
292,B0781VSXBP,202209,1910.0,74470.90,38.990000,Crucial,Internal Solid State Drives,MX500 SSD,4,09,2022-02-28,2,True,False
104,B003J5JB12,202126,3143.0,537421.57,170.990000,Crucial,Internal Solid State Drives,MX500 SSD,3,26,2021-06-28,6,True,False
127,B003J5JB12,202149,3251.0,546135.49,167.990000,Crucial,Internal Solid State Drives,MX500 SSD,3,49,2021-12-06,12,True,False
2258,B08FSZT2J7,202126,2571.0,195370.28,75.989996,Crucial,External Solid State Drives,X6,3,26,2021-06-28,6,True,False
2185,B08FSNKNSV,202126,1587.0,226131.63,142.490000,Crucial,External Solid State Drives,X6,3,26,2021-06-28,6,True,False
2455,B08W1KDM9K,202126,1714.0,496974.30,289.950000,Crucial,External Solid State Drives,X6,3,26,2021-06-28,6,True,False


In [183]:
df = pd.get_dummies(data=df, columns=['month'], drop_first = True)
"""drop_first=True is important to use, as it helps in reducing the extra column created during dummy variable creation. Hence it reduces the correlations created among dummy variables.
Lets say we have 3 types of values in Categorical column and we want to create dummy variable for that column. If one variable is not furnished and semi_furnished, then It is obvious unfurnished. So we do not need 3rd variable to identify the unfurnished. Example
Hence if we have categorical variable with n-levels, then we need to use n-1 columns to represent the dummy variables."""

'drop_first=True is important to use, as it helps in reducing the extra column created during dummy variable creation. Hence it reduces the correlations created among dummy variables.\nLets say we have 3 types of values in Categorical column and we want to create dummy variable for that column. If one variable is not furnished and semi_furnished, then It is obvious unfurnished. So we do not need 3rd variable to identify the unfurnished. Example\nHence if we have categorical variable with n-levels, then we need to use n-1 columns to represent the dummy variables.'

In [184]:
df

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,PRODUCT_GROUP,trend,Week,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
1263,B086BGWNY8,202102,1017.0,54348.48,53.440000,Crucial,Internal Solid State Drives,P2 SSD,3,02,...,0,0,0,0,0,0,0,0,0,0
1264,B086BGWNY8,202103,1064.0,57115.52,53.680000,Crucial,Internal Solid State Drives,P2 SSD,3,03,...,0,0,0,0,0,0,0,0,0,0
1265,B086BGWNY8,202104,1487.0,78796.13,52.990000,Crucial,Internal Solid State Drives,P2 SSD,3,04,...,0,0,0,0,0,0,0,0,0,0
1266,B086BGWNY8,202105,1136.0,60196.64,52.990000,Crucial,Internal Solid State Drives,P2 SSD,3,05,...,0,0,0,0,0,0,0,0,0,0
1267,B086BGWNY8,202106,782.0,45066.66,57.630000,Crucial,Internal Solid State Drives,P2 SSD,3,06,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,B08W1KDM9K,202221,52.0,18615.48,357.990000,Crucial,External Solid State Drives,X6,4,21,...,0,0,1,0,0,0,0,0,0,0
2497,B08W1KDM9K,202222,76.0,27093.24,356.490000,Crucial,External Solid State Drives,X6,4,22,...,0,0,1,0,0,0,0,0,0,0
2498,B08W1KDM9K,202223,64.0,22271.99,347.999844,Crucial,External Solid State Drives,X6,4,23,...,0,0,0,1,0,0,0,0,0,0
2499,B08W1KDM9K,202224,80.0,26800.00,335.000000,Crucial,External Solid State Drives,X6,4,24,...,0,0,0,1,0,0,0,0,0,0


In [185]:
df = pd.get_dummies(data=df, columns=['PRODUCT_GROUP'], drop_first = True)

In [186]:
df["Price-1"] = df.groupby(['ASIN'])['Price'].shift(1)
df["Price-2"] = df.groupby(['ASIN'])['Price'].shift(2)
df.dropna(subset=['Price-1',"Price-2"],inplace=True)
df.head()

,ASIN,Week ID,Volume,Estimated Sales,Price,Brand Name,Subcategory Name,trend,Week,week_date,...,month_9,month_10,month_11,month_12,PRODUCT_GROUP_MX500 SSD,PRODUCT_GROUP_P2 SSD,PRODUCT_GROUP_P5 Plus,PRODUCT_GROUP_X6,Price-1,Price-2
1265,B086BGWNY8,202104,1487.0,78796.13,52.99,Crucial,Internal Solid State Drives,3,04,2021-01-25,...,0,0,0,0,0,1,0,0,53.68,53.44
1266,B086BGWNY8,202105,1136.0,60196.64,52.99,Crucial,Internal Solid State Drives,3,05,2021-02-01,...,0,0,0,0,0,1,0,0,52.99,53.68
1267,B086BGWNY8,202106,782.0,45066.66,57.63,Crucial,Internal Solid State Drives,3,06,2021-02-08,...,0,0,0,0,0,1,0,0,52.99,52.99
1268,B086BGWNY8,202107,591.0,34419.84,58.24,Crucial,Internal Solid State Drives,3,07,2021-02-15,...,0,0,0,0,0,1,0,0,57.63,52.99
1269,B086BGWNY8,202108,455.0,27295.45,59.99,Crucial,Internal Solid State Drives,3,08,2021-02-22,...,0,0,0,0,0,1,0,0,58.24,57.63


In [187]:
## Put lag-prices next to the price column ##
#price
col = df.pop("Price")
df.insert(4, col.name, col)
pos_price=df.columns.get_loc('Price')
#p-1
col = df.pop("Price-1")
df.insert(pos_price+1, col.name, col)
#p-2
col = df.pop("Price-2")
df.insert(pos_price+2, col.name, col)
col=df.pop("week_date")
df.insert(1, col.name, col)
df=df.drop(columns=['Week ID', 'Week','Brand Name','Subcategory Name'])
#plot
df.head()

,ASIN,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,...,month_7,month_8,month_9,month_10,month_11,month_12,PRODUCT_GROUP_MX500 SSD,PRODUCT_GROUP_P2 SSD,PRODUCT_GROUP_P5 Plus,PRODUCT_GROUP_X6
1265,B086BGWNY8,2021-01-25,1487.0,78796.13,52.99,53.68,53.44,3,0,0,...,0,0,0,0,0,0,0,1,0,0
1266,B086BGWNY8,2021-02-01,1136.0,60196.64,52.99,52.99,53.68,3,1,0,...,0,0,0,0,0,0,0,1,0,0
1267,B086BGWNY8,2021-02-08,782.0,45066.66,57.63,52.99,52.99,3,1,0,...,0,0,0,0,0,0,0,1,0,0
1268,B086BGWNY8,2021-02-15,591.0,34419.84,58.24,57.63,52.99,3,1,0,...,0,0,0,0,0,0,0,1,0,0
1269,B086BGWNY8,2021-02-22,455.0,27295.45,59.99,58.24,57.63,3,1,0,...,0,0,0,0,0,0,0,1,0,0


In [188]:
df = df.sort_values(by=['ASIN','week_date'])

In [189]:
df

,ASIN,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,...,month_7,month_8,month_9,month_10,month_11,month_12,PRODUCT_GROUP_MX500 SSD,PRODUCT_GROUP_P2 SSD,PRODUCT_GROUP_P5 Plus,PRODUCT_GROUP_X6
2,B003J5JB12,2018-02-05,12.0,5999.88,499.99,499.99,515.35,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,B003J5JB12,2018-02-12,11.0,5499.89,499.99,499.99,499.99,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,B003J5JB12,2018-02-19,10.0,4999.90,499.99,499.99,499.99,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,B003J5JB12,2018-02-26,11.0,5499.89,499.99,499.99,499.99,0,1,0,...,0,0,0,0,0,0,1,0,0,0
6,B003J5JB12,2018-03-05,12.0,5999.88,499.99,499.99,499.99,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,B098WKQRDL,2022-05-23,213.0,51088.05,239.85,239.85,239.99,4,0,0,...,0,0,0,0,0,0,0,0,1,0
2627,B098WKQRDL,2022-05-30,270.0,61557.30,227.99,239.85,239.85,4,0,0,...,0,0,0,0,0,0,0,0,1,0
2628,B098WKQRDL,2022-06-06,287.0,65433.13,227.99,227.99,239.85,4,0,0,...,0,0,0,0,0,0,0,0,1,0
2629,B098WKQRDL,2022-06-13,307.0,69992.93,227.99,227.99,227.99,4,0,0,...,0,0,0,0,0,0,0,0,1,0


In [231]:
df.to_excel('Preprocessed_data2.xlsx')

In [168]:
pip install statsmodels

Looking in indexes: https://artifactory.micron.com/artifactory/api/pypi/zextpythonorg-pypi-rel-remote/simple



In [190]:
ASINSet = list(df.ASIN.unique())
ASINData = {}
colnames = [i for i in df.columns if i not in ["week_date","ASIN","Volume","Estimated Sales"]]
for i in ASINSet:
  df_i = df[df.ASIN == i]
  ASINData[i] = {'X': df_i[colnames].values,
                'y': df_i.Volume.values}

In [233]:
ASINData

{'B003J5JB12': {'X': array([[499.99, 499.99, 515.35, ..., 0, 0, 0],
         [499.99, 499.99, 499.99, ..., 0, 0, 0],
         [499.99, 499.99, 499.99, ..., 0, 0, 0],
         ...,
         [179.99, 194.18, 193.35, ..., 0, 0, 0],
         [179.9899999, 179.99, 194.18, ..., 0, 0, 0],
         [177.39, 179.9899999, 179.99, ..., 0, 0, 0]], dtype=object),
  'y': array([  12.,   11.,   10.,   11.,   12.,   22.,   29.,    8.,   13.,
            7.,    5.,    4.,   14.,   23.,   29.,   13.,   22.,   11.,
           18.,    9.,   12.,   13.,   12.,   42.,   59.,   14.,   37.,
           23.,   50.,    8.,   13.,    6.,   19.,   41.,   13.,    9.,
           10.,    9.,   19.,   14.,   15.,   16.,  210.,  150.,  185.,
           42.,   50.,   71.,  117.,  107.,   86.,   69.,   58.,   63.,
           45.,   55.,   58.,   44.,   50.,   19.,  155.,   99.,   40.,
           32.,   17.,   28.,   22.,   35.,   34.,   54.,   64.,   50.,
           46.,   68.,  128.,  896.,  194.,  101.,  103.,  200.,  

In [191]:
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in ASINSet:
  
  X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))]) #split for X
  y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

In [192]:
import time
tZero=time.time()

X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

from sklearn.linear_model import LinearRegression
model_cen = LinearRegression().fit(X_cen_train, y_train)

from sklearn.metrics import r2_score, mean_squared_error
print('OOS R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('OOS MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec.")

OOS R2: 0.248
OOS MSE: 816497.468
Time to compute: 0.004  sec.


In [193]:
res=pd.DataFrame(index=['R2'])
res['centralized']=[r2_score(y_test, model_cen.predict(X_cen_test))]
res

,centralized
R2,0.248361


In [237]:
X_dict['B0781VSXBP']['train'].astype(float)
ty=pd.DataFrame(X_dict['B0781VSXBP']['train'].astype(float))

In [238]:
ty

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,79.99,79.99,79.99,49.99,79.99,79.99,1.101208e-06,35.99,79.99,52.560392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,79.99,79.99,79.99,49.99,79.99,79.99,1.101208e-06,35.99,79.99,52.560392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,79.99,79.99,79.99,49.99,79.99,79.99,1.101208e-06,35.99,79.99,52.560392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,79.99,79.99,79.99,49.99,79.99,79.99,1.101208e-06,35.99,79.99,52.560392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,79.99,79.99,79.99,49.99,79.99,79.99,1.101208e-06,35.99,79.99,52.560392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,44.95,44.95,49.95,49.99,44.95,49.95,2.665519e+00,35.99,79.99,52.560392,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,49.95,44.95,44.95,49.99,49.95,49.95,7.786718e-07,35.99,79.99,52.560392,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,49.95,49.95,44.95,49.99,49.95,49.95,7.786718e-07,35.99,79.99,52.560392,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,46.99,49.95,49.95,49.99,46.99,49.95,1.444332e+00,35.99,79.99,52.560392,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [194]:
from statsmodels.regression.linear_model import OLS
tZero=time.time()

y_pred = []
ASINModels = {}

for i in ASINSet:
 #one model for each item, fitted on training set
 model_i = OLS(y_dict[i]['train'].astype(float), X_dict[i]['train'].astype(float), hasconst = False)
 ASINModels[i] = model_i.fit()

 #compute and concatenate prediction of the model i on item i
 y_pred += list(ASINModels[i].predict(X_dict[i]['test'].astype(float)))


#computing overall performance metrics on y_pred and y_test:
print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('OOS MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: -0.107
OOS MSE: 1202351.321
Time to compute: 0.009  sec


In [64]:
res['decentralized']=[r2_score(y_test, np.array(y_pred))]
res

,centralized,decentralized
R2,0.248361,0.28325


In [195]:
def forward_stepwise_selection(data,p,metric='aic'):
    """

    Perform selection of p features on a dataset 


    p: highest number of features used to train each model
    metric: aic (default) or bic

    Returns best trained model and list of features selected

    """

    available_features = set(data.columns)

    selected = []
    current_score, best_new_score = 1000, 1000

    overall_selected = []
    while available_features and current_score == best_new_score and len(selected)<p:

        scores_with_candidates = []
        for candidate in available_features:
            
            selected_candidates = selected.copy()
            
            selected_candidates.append(candidate)
            # print(selected_candidates)
            if metric == 'aic':
              score = OLS(y_train_primer, X_train_primer[selected_candidates]).fit().aic
            elif metric == 'bic':
              score = OLS(y_train_primer, X_train_primer[selected_candidates]).fit().bic

            else:
              print('Metric not supported')
              break
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort(reverse=True)
        best_new_score, best_candidate = scores_with_candidates.pop()

        available_features.remove(best_candidate) #remove the selected feature from available features
        selected.append(best_candidate) #add it to our list of features selected

        # display results at each stage
        print(str(len(selected))+'. :',selected,best_new_score)

        if current_score > best_new_score: #if this model generates an improvement we record it

            current_score = best_new_score
            overall_selected = selected.copy()
            

    model = OLS(y_train_primer, X_train_primer[overall_selected]).fit()
    return(model,overall_selected)

In [196]:
import numpy as np
data = df[df.ASIN=='B08FSNKNSV'].sort_values(by=["week_date"])
colnames = [i for i in data.columns if i not in ["week_date","ASIN","Volume","Estimated Sales"]]
data

X_primer = data[colnames].astype(float)
y_primer = data.Volume.astype(float)

X_train_primer,X_test_primer = np.split(X_primer, [int(.68*len(X_primer))])
y_train_primer, y_test_primer = np.split(y_primer, [int(.68*len(y_primer))])

In [197]:
model,selected = forward_stepwise_selection(X_train_primer,15,'aic')

print(selected)
print(model.aic)

1. : ['Price-2'] 680.3310495241948
2. : ['Price-2', 'Price'] 662.7530767982381
3. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6'] 651.6104119515463
4. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6'] 644.7320740107059
5. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1'] 643.0481420658515
6. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1', 'month_10'] 641.8248085917556
7. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1', 'month_10', 'month_7'] 641.3003637870769
8. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1', 'month_10', 'month_7', 'PRODUCT_GROUP_MX500 SSD'] 641.3003637870769
9. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1', 'month_10', 'month_7', 'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD'] 641.3003637870769
10. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'Price-1', 'month_10', 'month_7', 'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD', 'PRODUCT_GROUP_P5 Plus'] 641.3003637870769
11. : [

In [198]:
model = OLS(y_train_primer, X_train_primer[selected])
model = model.fit()
y_pred_primer = list(model.predict(X_test_primer[selected]))

print('OOS R2:',round(r2_score(y_test_primer, np.array(y_pred_primer)),3))
print('OOS MSE:', round(mean_squared_error(y_test_primer, np.array(y_pred_primer)),3))

OOS R2: -3.26
OOS MSE: 26441.845


In [199]:
model,selected = forward_stepwise_selection(X_train_primer,12,'bic')

print(selected)
print(model.bic)

1. : ['Price-2'] 682.2022505351026
2. : ['Price-2', 'Price'] 666.4954788200539
3. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6'] 657.22401498427
4. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6'] 652.2168780543375
5. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'PRODUCT_GROUP_MX500 SSD'] 652.2168780543375
6. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD'] 652.2168780543375
7. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD', 'PRODUCT_GROUP_P5 Plus'] 652.2168780543375
8. : ['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6', 'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD', 'PRODUCT_GROUP_P5 Plus', 'Price-1'] 652.4041471203909
['Price-2', 'Price', 'PRODUCT_GROUP_X6', 'month_6']
652.2168780543375


In [200]:
model = OLS(y_train_primer, X_train_primer[selected])
model = model.fit()
y_pred_primer = list(model.predict(X_test_primer[selected]))
from sklearn.metrics import r2_score, mean_squared_error
print('OOS R2:',round(r2_score(y_test_primer, np.array(y_pred_primer)),3))
print('OOS MSE:', round(mean_squared_error(y_test_primer, np.array(y_pred_primer)),3))

OOS R2: -6.511
OOS MSE: 46624.898


In [201]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(alpha=0.95, fit_intercept=True, tol=0.00001,
          max_iter=10000, positive=True,selection='cyclic')
lasso_model.fit(X_train_primer,y_train_primer)

Lasso(alpha=0.95, max_iter=10000, positive=True, tol=1e-05)

In [202]:
lasso_df = pd.DataFrame(list(zip(X_primer.columns,lasso_model.coef_)),columns=["feature","coefficient"])
lasso_df[lasso_df.coefficient !=0]

,feature,coefficient
1,Price-1,1.622898
2,Price-2,1.736386
3,trend,228.446934
5,month_3,49.651841
6,month_4,35.481248
7,month_5,23.947153
8,month_6,317.234583
9,month_7,37.067205
10,month_8,84.573229
12,month_10,515.467919


In [203]:
y_pred_primer = list(lasso_model.predict(X_test_primer))
print('OOS R2:',round(r2_score(y_test_primer, np.array(y_pred_primer)),3))
print('OOS MSE:', round(mean_squared_error(y_test_primer, np.array(y_pred_primer)),3))

OOS R2: -2.833
OOS MSE: 23793.261


In [204]:
tZero=time.time()

y_pred = []

ASINModelsLasso = {}

for i in ASINSet:
   
  model_i = linear_model.Lasso(alpha=0.95, fit_intercept=True, tol=0.00001,
          max_iter=10000, positive=True)
 
  ASINModelsLasso[i] = model_i.fit(X_dict[i]["train"].astype(float),y_dict[i]["train"].astype(float))
  y_pred += list(ASINModelsLasso[i].predict(X_dict[i]['test'].astype(float)))

print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('OOS MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.315
OOS MSE: 744543.386
Time to compute: 0.009  sec


In [205]:
res['decentralized_lasso']=[r2_score(y_test, np.array(y_pred))]
res

,centralized,decentralized_lasso
R2,0.248361,0.314599


In [206]:
from sklearn.linear_model import Ridge
tZero=time.time()

y_pred = []

skuModelsRidge = {}

for i in ASINSet:
   
  model_i = Ridge(alpha=0.95, fit_intercept=True, tol=0.00001,
          max_iter=100000,solver='auto')
 
  skuModelsRidge[i] = model_i.fit(X_dict[i]["train"],y_dict[i]["train"])
  y_pred += list(skuModelsRidge[i].predict(X_dict[i]['test']))

print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('OOS MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.409
OOS MSE: 642243.597
Time to compute: 0.011  sec


In [207]:
res['decentralized_ridge']=[r2_score(y_test, np.array(y_pred))]
res

,centralized,decentralized_lasso,decentralized_ridge
R2,0.248361,0.314599,0.408773


In [208]:
from sklearn.linear_model import ElasticNet
tZero=time.time()

y_pred = []
skuModelsElastic = {}

for i in ASINSet:
  elastic = ElasticNet(alpha= 0.05,l1_ratio=0.3, max_iter=100000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
  skuModelsElastic[i] = elastic.fit(X_dict[i]["train"],y_dict[i]["train"])
  y_pred += list(skuModelsElastic[i].predict(X_dict[i]['test']))

print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('OOS MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

t = time.time()-tZero
print("Time to compute",round(t,3)," sec")

OOS R2: 0.421
OOS MSE: 629180.241
Time to compute 0.014  sec


In [209]:
res['decentralized_elasticnet']=[r2_score(y_test, np.array(y_pred))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet
R2,0.248361,0.314599,0.408773,0.420798


In [210]:
df_log_lin = df.copy()
df_log_lin['Price']=np.log(df_log_lin['Price'])
df_log_lin['Price-1']=np.log(df_log_lin['Price-1'])
df_log_lin['Price-2']=np.log(df_log_lin['Price-2'])
df_log_lin.head()

,ASIN,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,...,month_7,month_8,month_9,month_10,month_11,month_12,PRODUCT_GROUP_MX500 SSD,PRODUCT_GROUP_P2 SSD,PRODUCT_GROUP_P5 Plus,PRODUCT_GROUP_X6
2,B003J5JB12,2018-02-05,12.0,5999.88,6.214588,6.214588,6.244846,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,B003J5JB12,2018-02-12,11.0,5499.89,6.214588,6.214588,6.214588,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,B003J5JB12,2018-02-19,10.0,4999.90,6.214588,6.214588,6.214588,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,B003J5JB12,2018-02-26,11.0,5499.89,6.214588,6.214588,6.214588,0,1,0,...,0,0,0,0,0,0,1,0,0,0
6,B003J5JB12,2018-03-05,12.0,5999.88,6.214588,6.214588,6.214588,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [211]:
ASINData = {}
colnames = [i for i in df.columns if i not in ["week_date","ASIN","Volume","Estimated Sales"]] 
for i in ASINSet:
  df_i = df_log_lin[df_log_lin.ASIN == i]
  ASINData[i] = {'X': df_i[colnames].values,
                'y': df_i.Volume.values}

In [320]:
ASINData

{'B003J5JB12': {'X': array([[6.2145880982221895, 6.2145880982221895, 6.244846281482801, ...,
          0, 0, 0],
         [6.2145880982221895, 6.2145880982221895, 6.2145880982221895, ...,
          0, 0, 0],
         [6.2145880982221895, 6.2145880982221895, 6.2145880982221895, ...,
          0, 0, 0],
         ...,
         [5.192901293791388, 5.268785563941999, 5.264502018053306, ..., 0,
          0, 0],
         [5.192901293235801, 5.192901293791388, 5.268785563941999, ..., 0,
          0, 0],
         [5.178350698500848, 5.192901293235801, 5.192901293791388, ..., 0,
          0, 0]], dtype=object),
  'y': array([  12.,   11.,   10.,   11.,   12.,   22.,   29.,    8.,   13.,
            7.,    5.,    4.,   14.,   23.,   29.,   13.,   22.,   11.,
           18.,    9.,   12.,   13.,   12.,   42.,   59.,   14.,   37.,
           23.,   50.,    8.,   13.,    6.,   19.,   41.,   13.,    9.,
           10.,    9.,   19.,   14.,   15.,   16.,  210.,  150.,  185.,
           42.,   50.,   7

In [212]:
# testing data dictionary
X_dict = {}
y_dict = {}

ASINModels = {}
y_pred = []
y_test = []
y_train = []

for i in ASINSet:

    X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))])
    y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))])

    X_dict[i] = {'train': X_train_i, 'test': X_test_i}
    y_dict[i] = {'train': y_train_i, 'test': y_test_i}
    
    model_i = OLS(y_train_i.astype(float), X_train_i.astype(float))
    ASINModels[i] = model_i.fit()
    y_pred += list(ASINModels[i].predict(X_test_i.astype(float)))

    y_test += list(y_test_i)
    y_train += list(y_train_i)

y_train = np.array(y_train)
y_test = np.array(y_test)

print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))

OOS R2: -0.044


In [213]:
res['decentralized_log_lin']=[r2_score(y_test, np.array(y_pred))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet,decentralized_log_lin
R2,0.248361,0.314599,0.408773,0.420798,-0.044127


In [214]:
sales_fe_ASIN = df.copy()
sales_fe_ASIN = pd.get_dummies(data=sales_fe_ASIN, columns=['ASIN'])
sales_fe_ASIN["ASIN"] = df["ASIN"] 
sales_fe_ASIN.head(5)

,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,month_4,...,ASIN_B07YD579WM,ASIN_B086BGWNY8,ASIN_B089DNM8LR,ASIN_B08FSNKNSV,ASIN_B08FSZT2J7,ASIN_B08GVDNTGJ,ASIN_B08W1F8YGJ,ASIN_B08W1KDM9K,ASIN_B098WKQRDL,ASIN
2,2018-02-05,12.0,5999.88,499.99,499.99,515.35,0,1,0,0,...,0,0,0,0,0,0,0,0,0,B003J5JB12
3,2018-02-12,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,B003J5JB12
4,2018-02-19,10.0,4999.90,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,B003J5JB12
5,2018-02-26,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,B003J5JB12
6,2018-03-05,12.0,5999.88,499.99,499.99,499.99,0,0,1,0,...,0,0,0,0,0,0,0,0,0,B003J5JB12


In [215]:
ASINSet = list(sales_fe_ASIN.ASIN.unique()) #the SKU numbers do not change
ASINData = {}
colnames = [i for i in sales_fe_ASIN.columns if i not in ["week_date","ASIN","Volume","Estimated Sales"]]
for i in ASINSet:
  df_i = sales_fe_ASIN[sales_fe_ASIN.ASIN == i]
  ASINData[i] = {'X': df_i[colnames].values,
                'y': df_i.Volume.values}

In [216]:
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in ASINSet:
  
  X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))]) #split for X
  y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

In [95]:
X_dict[ASINSet[0]]['train'] 

array([[499.99, 499.99, 515.35, ..., 0, 0, 0],
       [499.99, 499.99, 499.99, ..., 0, 0, 0],
       [499.99, 499.99, 499.99, ..., 0, 0, 0],
       ...,
       [209.95, 209.95, 209.95, ..., 0, 0, 0],
       [170.9899999, 209.95, 209.95, ..., 0, 0, 0],
       [209.95, 170.9899999, 209.95, ..., 0, 0, 0]], dtype=object)

In [101]:
ASINData['B003J5JB12']["X"]

array([[499.99, 499.99, 515.35, ..., 0, 0, 0],
       [499.99, 499.99, 499.99, ..., 0, 0, 0],
       [499.99, 499.99, 499.99, ..., 0, 0, 0],
       ...,
       [179.99, 194.18, 193.35, ..., 0, 0, 0],
       [179.9899999, 179.99, 194.18, ..., 0, 0, 0],
       [177.39, 179.9899999, 179.99, ..., 0, 0, 0]], dtype=object)

In [217]:
import time
tZero=time.time()

X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

model_cen = LinearRegression().fit(X_cen_train, y_train)
print('OOS R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.412
MSE: 638616.712
Time to compute: 0.005  sec


In [104]:
import time
tZero=time.time()

X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((np.array(X_cen_train, dtype=float), np.array((X_dict[i]['train']),dtype=float)), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((np.array(X_cen_test, dtype=float), np.array((X_dict[i]['test']),dtype=float)), axis = 0)

from sklearn.linear_model import LinearRegression
model_cen = LinearRegression().fit(np.array(X_cen_train, dtype=float), np.array(y_train, dtype=float))

from sklearn.metrics import r2_score, mean_squared_error
print('OOS R2:', round(r2_score(np.array(y_test, dtype=float), model_cen.predict(np.array(X_cen_test, dtype=float))),3))  
print('OOS MSE:', round(mean_squared_error(np.array(y_test, dtype=float), model_cen.predict(np.array(X_cen_test, dtype=float))),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec.")

OOS R2: 0.412
OOS MSE: 638616.712
Time to compute: 0.007  sec.


In [218]:
fixed_effect_sku = pd.DataFrame(zip(colnames,model_cen.coef_),columns=["feature","coef"])
fixed_effect_sku

,feature,coef
0,Price,-6.779117
1,Price-1,3.976243
2,Price-2,4.000148
3,trend,368.350602
4,month_2,-180.225164
5,month_3,-11.417939
6,month_4,-71.359405
7,month_5,69.425921
8,month_6,276.923636
9,month_7,80.601951


In [219]:
res['Centralized - SKU FE']=[r2_score(y_test, model_cen.predict(X_cen_test))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet,decentralized_log_lin,Centralized - SKU FE
R2,0.248361,0.314599,0.408773,0.420798,-0.044127,0.412112


Centralized-Fixed Price

In [244]:
sales_fe_price = df.copy()
for i in ASINSet[1:]:
  sales_fe_price["price_fixed_effect_"+str(i)] = sales_fe_ASIN["Price"]*sales_fe_ASIN["ASIN_"+str(i)]
sales_fe_price.head(5)

,ASIN,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,...,price_fixed_effect_B0786QNS9B,price_fixed_effect_B07YD579WM,price_fixed_effect_B086BGWNY8,price_fixed_effect_B089DNM8LR,price_fixed_effect_B08FSNKNSV,price_fixed_effect_B08FSZT2J7,price_fixed_effect_B08GVDNTGJ,price_fixed_effect_B08W1F8YGJ,price_fixed_effect_B08W1KDM9K,price_fixed_effect_B098WKQRDL
2,B003J5JB12,2018-02-05,12.0,5999.88,499.99,499.99,515.35,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B003J5JB12,2018-02-12,11.0,5499.89,499.99,499.99,499.99,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B003J5JB12,2018-02-19,10.0,4999.90,499.99,499.99,499.99,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B003J5JB12,2018-02-26,11.0,5499.89,499.99,499.99,499.99,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,B003J5JB12,2018-03-05,12.0,5999.88,499.99,499.99,499.99,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
ASINSet

['B003J5JB12',
 'B0781VSXBP',
 'B078211KBB',
 'B0786QNS9B',
 'B07YD579WM',
 'B086BGWNY8',
 'B089DNM8LR',
 'B08FSNKNSV',
 'B08FSZT2J7',
 'B08GVDNTGJ',
 'B08W1F8YGJ',
 'B08W1KDM9K',
 'B098WKQRDL']

In [221]:
sales_fe_price[sales_fe_price.ASIN=='B086BGWNY8'][["week_date","ASIN","price_fixed_effect_B086BGWNY8","price_fixed_effect_B0786QNS9B"]]

,week_date,ASIN,price_fixed_effect_B086BGWNY8,price_fixed_effect_B0786QNS9B
1265,2021-01-25,B086BGWNY8,52.99,0.0
1266,2021-02-01,B086BGWNY8,52.99,0.0
1267,2021-02-08,B086BGWNY8,57.63,0.0
1268,2021-02-15,B086BGWNY8,58.24,0.0
1269,2021-02-22,B086BGWNY8,59.99,0.0
...,...,...,...,...
1330,2022-05-23,B086BGWNY8,49.19,0.0
1331,2022-05-30,B086BGWNY8,55.99,0.0
1332,2022-06-06,B086BGWNY8,54.99,0.0
1333,2022-06-13,B086BGWNY8,54.99,0.0


In [245]:
ASINData = {}
colnames = [i for i in sales_fe_price.columns if i not in ["week_date","ASIN","Volume","Estimated Sales","Price"]] #remove price from the features

for i in ASINSet:
 df_i = sales_fe_price[sales_fe_price.ASIN == i]
 ASINData[i] = {'X': df_i[colnames].values,
               'y': df_i.Volume.values}

In [250]:
df_i[colnames]

,Price-1,Price-2,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,...,price_fixed_effect_B0786QNS9B,price_fixed_effect_B07YD579WM,price_fixed_effect_B086BGWNY8,price_fixed_effect_B089DNM8LR,price_fixed_effect_B08FSNKNSV,price_fixed_effect_B08FSZT2J7,price_fixed_effect_B08GVDNTGJ,price_fixed_effect_B08W1F8YGJ,price_fixed_effect_B08W1KDM9K,price_fixed_effect_B098WKQRDL
2592,367.990000,367.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,367.990000
2593,367.990000,367.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,367.990000
2594,367.990000,367.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.020000
2595,341.020000,367.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,319.990000
2596,319.990000,341.020000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,309.000000
2597,309.000000,319.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.920000
2598,301.920000,309.000000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299.990000
2599,299.990000,301.920000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,307.000000
2600,307.000000,299.990000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,328.990000
2601,328.990000,307.000000,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,299.990000


In [248]:
dq2 = pd.DataFrame(ASINData['B0781VSXBP']["X"])
dq2.to_excel("Error3.xlsx")

In [223]:
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in ASINSet:
  
  X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))]) #split for X
  y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

In [224]:

X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

model_cen = LinearRegression().fit(X_cen_train, y_train)
print('OOS R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.213
MSE: 854688.375
Time to compute: 38.069  sec


In [225]:
res['Centralized - Price FE']=[r2_score(y_test, model_cen.predict(X_cen_test))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet,decentralized_log_lin,Centralized - SKU FE,Centralized - Price FE
R2,0.248361,0.314599,0.408773,0.420798,-0.044127,0.412112,0.213204


In [226]:
sales_fe_sku_price = sales_fe_ASIN.copy()
for i in ASINSet[1:]:
  sales_fe_sku_price["price_fixed_effect_"+str(i)] = sales_fe_sku_price["Price"]*sales_fe_sku_price["ASIN_"+str(i)]
sales_fe_sku_price.head(5)

,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,month_4,...,price_fixed_effect_B0786QNS9B,price_fixed_effect_B07YD579WM,price_fixed_effect_B086BGWNY8,price_fixed_effect_B089DNM8LR,price_fixed_effect_B08FSNKNSV,price_fixed_effect_B08FSZT2J7,price_fixed_effect_B08GVDNTGJ,price_fixed_effect_B08W1F8YGJ,price_fixed_effect_B08W1KDM9K,price_fixed_effect_B098WKQRDL
2,2018-02-05,12.0,5999.88,499.99,499.99,515.35,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-02-12,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-02-19,10.0,4999.90,499.99,499.99,499.99,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2018-02-26,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2018-03-05,12.0,5999.88,499.99,499.99,499.99,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
ASINData = {}
colnames = [i for i in sales_fe_sku_price.columns if i not in ["week_date","ASIN","Volume","Estimated Sales","Price"]] #remove price from the features

for i in ASINSet:
 df_i = sales_fe_sku_price[sales_fe_sku_price.ASIN == i]
 ASINData[i] = {'X': df_i[colnames].values,
               'y': df_i.Volume.values}

In [228]:
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in ASINSet:
  
  X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))]) #split for X
  y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

In [229]:
tZero=time.time()
X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

model_cen = LinearRegression(fit_intercept=True).fit(X_cen_train, y_train)
print('OOS R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.337
MSE: 720724.436
Time to compute: 0.005  sec


In [230]:
res['Centralized - SKU + Price FE']=[r2_score(y_test, model_cen.predict(X_cen_test))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet,decentralized_log_lin,Centralized - SKU FE,Centralized - Price FE,Centralized - SKU + Price FE
R2,0.248361,0.314599,0.408773,0.420798,-0.044127,0.412112,0.213204,0.336526


In [231]:
colnames_to_fix = [i for i in df.columns if i not in ["week_date","ASIN","Volume","Estimated Sales",
                                                         'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                                                         'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]
colnames_to_fix

['Price',
 'Price-1',
 'Price-2',
 'trend',
 'PRODUCT_GROUP_MX500 SSD',
 'PRODUCT_GROUP_P2 SSD',
 'PRODUCT_GROUP_P5 Plus',
 'PRODUCT_GROUP_X6']

In [232]:
sales_seas = sales_fe_ASIN.copy()
sales_seas.columns

Index(['week_date', 'Volume', 'Estimated Sales', 'Price', 'Price-1', 'Price-2',
       'trend', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'PRODUCT_GROUP_MX500 SSD', 'PRODUCT_GROUP_P2 SSD',
       'PRODUCT_GROUP_P5 Plus', 'PRODUCT_GROUP_X6', 'ASIN_B003J5JB12',
       'ASIN_B0781VSXBP', 'ASIN_B078211KBB', 'ASIN_B0786QNS9B',
       'ASIN_B07YD579WM', 'ASIN_B086BGWNY8', 'ASIN_B089DNM8LR',
       'ASIN_B08FSNKNSV', 'ASIN_B08FSZT2J7', 'ASIN_B08GVDNTGJ',
       'ASIN_B08W1F8YGJ', 'ASIN_B08W1KDM9K', 'ASIN_B098WKQRDL', 'ASIN'],
      dtype='object')

In [233]:
for feature in colnames_to_fix:
  for i in ASINSet[1:]:
    sales_seas[str(feature)+"_fixed_effect_"+str(i)] = sales_seas[feature]*sales_seas["ASIN_"+str(i)]
sales_seas.head(5)

,week_date,Volume,Estimated Sales,Price,Price-1,Price-2,trend,month_2,month_3,month_4,...,PRODUCT_GROUP_X6_fixed_effect_B0786QNS9B,PRODUCT_GROUP_X6_fixed_effect_B07YD579WM,PRODUCT_GROUP_X6_fixed_effect_B086BGWNY8,PRODUCT_GROUP_X6_fixed_effect_B089DNM8LR,PRODUCT_GROUP_X6_fixed_effect_B08FSNKNSV,PRODUCT_GROUP_X6_fixed_effect_B08FSZT2J7,PRODUCT_GROUP_X6_fixed_effect_B08GVDNTGJ,PRODUCT_GROUP_X6_fixed_effect_B08W1F8YGJ,PRODUCT_GROUP_X6_fixed_effect_B08W1KDM9K,PRODUCT_GROUP_X6_fixed_effect_B098WKQRDL
2,2018-02-05,12.0,5999.88,499.99,499.99,515.35,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-02-12,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-02-19,10.0,4999.90,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2018-02-26,11.0,5499.89,499.99,499.99,499.99,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2018-03-05,12.0,5999.88,499.99,499.99,499.99,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
sales_seas.to_excel("Weird.xlsx")

In [234]:
ASINData = {}
colnames = [i for i in sales_seas.columns if i not in ["week_date","ASIN","Volume","Estimated Sales"]] #remove price from the features

for i in ASINSet:
 df_i = sales_seas[sales_seas.ASIN == i]
 ASINData[i] = {'X': df_i[colnames].values,
               'y': df_i.Volume.values}

In [235]:
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in ASINSet:
  
  X_train_i,X_test_i = np.split(ASINData[i]["X"], [int(.68*len(ASINData[i]["X"]))]) #split for X
  y_train_i,y_test_i = np.split(ASINData[i]["y"], [int(.68*len(ASINData[i]["y"]))]) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

In [238]:
dq.to_excel("Error2.xlsx")

In [239]:
tZero=time.time()
X_cen_train = X_dict[ASINSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[ASINSet[0]]['test']

for i in ASINSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((np.array(X_cen_train, dtype=float), np.array((X_dict[i]['train']),dtype=float)), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((np.array(X_cen_test, dtype=float), np.array((X_dict[i]['test']),dtype=float)), axis = 0)

from sklearn.linear_model import LinearRegression
model_cen = LinearRegression(fit_intercept=True).fit(np.array(X_cen_train, dtype=float), np.array(y_train, dtype=float))

from sklearn.metrics import r2_score, mean_squared_error
print('OOS R2:', round(r2_score(np.array(y_test, dtype=float), model_cen.predict(np.array(X_cen_test, dtype=float))),3))  
print('OOS MSE:', round(mean_squared_error(np.array(y_test, dtype=float), model_cen.predict(np.array(X_cen_test, dtype=float))),3))

t = time.time()-tZero
print("Time to compute:",round(t,3)," sec")

OOS R2: 0.391
OOS MSE: 661868.947
Time to compute: 0.014  sec


In [251]:
res['Decentralized - Seasonality']=[r2_score(y_test, model_cen.predict(X_cen_test))]
res

,centralized,decentralized_lasso,decentralized_ridge,decentralized_elasticnet,decentralized_log_lin,Centralized - SKU FE,Centralized - Price FE,Centralized - SKU + Price FE,Decentralized - Seasonality
R2,0.248361,0.314599,0.408773,0.420798,-0.044127,0.412112,0.213204,0.336526,0.390706
